- For my final project, I will be analyzing how the number of certain cancer-risk SNPs correlate with specific ethnic groups.
- My specific research question is as followed: `Are the genotypic frequencies of cancer-risk SNPs different in certain ethnic groups compared to others?`


My analysis is follows:
- For each ethnic group, I will calculate the frequency of homozygous recessive (0|0),
homozygous dominate (1|1), and heterozygous (0|1, 1|0) for the cancer-risk SNPs.
- Once the frequency and percentages are calculated for each group, I will perform a
statistical test to determine if the differences in SNPs frequency between ethnic groups
statistically significant.
- If there is time, based on the genotypic frequencies, I will calculate if certain ethnic
groups are at a greater risk to developing certain cancers compared to other groups.

In [32]:
import csv

# Get Human IDs to Gene Frequencies

In [33]:
def getGeneData_and_HumanIDs(data_File):
    # Open the CSV file with the open() function
    # This returns a file object that you can use to read the file
    with open(data_File) as csvfile:
        # Use the csv.reader() function to read the file
        # This returns an iterable object that you can loop over
        data = csv.reader(csvfile)

        rawRowDataAsList = []

        # Loop over the rows in the file
        for row in data:
            rawRowDataAsList.append(row)

        # GENE DATA
        rawParsedGeneData = rawRowDataAsList[20][0].split("\t")
        geneData = rawParsedGeneData[9: len(rawParsedGeneData)-1]

        # HUMAN IDs 
        rawHumanIDs = rawRowDataAsList[19][0].split("\t")
        # print("rawHumanIDsParsed\n",rawHumanIDsParsed)

        humanIDs = rawHumanIDs[9: len(rawHumanIDs)-1]
        # print("humanIDs\n", humanIDs)
        # print(len(humanIDs))

    return (humanIDs, geneData)



In [34]:
humanIDs, geneData = getGeneData_and_HumanIDs('rawData/rs4713266.csv')

In [35]:
def get_humanID_to_genotype(humanIDs, geneData):
    humanID_to_genotype = {}
    
    for i in range(0, len(geneData)):

        if geneData[i] == "0|0":
            humanID_to_genotype[humanIDs[i]] = 0
        elif geneData[i] == "1|0" or geneData[i] == "0|1":
            humanID_to_genotype[humanIDs[i]] = 1
        elif geneData[i] == "1|1":
            humanID_to_genotype[humanIDs[i]] = 2

    return humanID_to_genotype


In [36]:
humanID_to_genotype = get_humanID_to_genotype(humanIDs, geneData)
# print(humanID_to_genotype)

In [37]:
def get_humanID_to_PopulationCode(data_File):
    # Open the CSV file with the open() function
    # This returns a file object that you can use to read the file
    with open(data_File) as csvfile:
        # Use the csv.reader() function to read the file
        # This returns an iterable object that you can loop over
        data = csv.reader(csvfile)

        rawRowDataAsList = []

        # Loop over the rows in the file
        for row in data:
            rawRowDataAsList.append(row)

        humanIDInfoHeader = rawRowDataAsList[0][0].split("\t")
        # print("humanIDInfoHeader\n", humanIDInfoHeader)
        del rawRowDataAsList[0]

        humanInfoParsed = []

        for rawHumanInfo in rawRowDataAsList:
            singleHumanInfoParsed = rawHumanInfo[0].split("\t")
            humanInfoParsed.append(singleHumanInfoParsed)
        # print("humanInfoParsed\n", humanInfoParsed)

        humanID_to_PopulationCode = {}

        for humanInfo in humanInfoParsed:
            humanID = humanInfo[0]
            populationCode = humanInfo[3]
            # humanID_and_PopulationCode.append([humanID, populationCode])
            humanID_to_PopulationCode[humanID] = populationCode

        # print("humanID_and_PopulationCode\n", humanID_to_PopulationCode)

    return humanID_to_PopulationCode



In [84]:
humanID_to_PopulationCode = get_humanID_to_PopulationCode('rawData/igsr_samples.tsv')
print(len(humanID_to_PopulationCode))

4978


# Merge Population Codes with Gene Frequencies at Corresponding Human IDs

In [85]:
def get_humanID_to_PopCode_Genotype(humanID_to_genotype, humanID_to_PopulationCode):

    humanID_to_PopCode_Genotype = {}

    for key in humanID_to_genotype.keys():
        # print("Ran outerloop...", key)
        if key in humanID_to_PopulationCode.keys():
            # print("Ran innerloop...", key)
            # print(humanID_to_PopulationCode.get(key), humanID_to_geneFrequency.get(key))
            humanID_to_PopCode_Genotype[key] = [humanID_to_PopulationCode.get(key), humanID_to_genotype.get(key)]

    # print("populationCode_to_geneFrequency\n", populationCode_to_geneFrequency)

    return humanID_to_PopCode_Genotype

In [100]:
humanID_to_PopCode_Genotype = get_humanID_to_PopCode_Genotype(humanID_to_genotype, humanID_to_PopulationCode)
# print(humanID_to_PopCode_Genotype)

In [114]:
def get_populationCode_totalGeneFrequencies(humanID_to_PopCode_Genotype):
    populationCode_to_geneFrequencies = {}

    for value in humanID_to_PopCode_Genotype.values():

        population_Code = value[0]
        genoType = value[1]

        if genoType == 0:
            genoTypeList = [1, 0, 0]
        elif genoType == 1:
            genoTypeList = [0, 1, 0]
        elif genoType == 2:
            genoTypeList = [0, 0, 1]

        if population_Code not in populationCode_to_geneFrequencies:
            populationCode_to_geneFrequencies[population_Code] = genoTypeList

        elif population_Code in populationCode_to_geneFrequencies:

            currentGenoTypeList = populationCode_to_geneFrequencies.get(population_Code)

            # Use a list comprehension to add the elements from the two lists together
            totalGenoTypeList = [x + y for x, y in zip(currentGenoTypeList, genoTypeList)]

            populationCode_to_geneFrequencies[population_Code] = totalGenoTypeList

    return populationCode_to_geneFrequencies


In [115]:
populationCode_to_geneFrequencies = get_populationCode_totalGeneFrequencies(humanID_to_PopCode_Genotype)
print("populationCode_to_geneFrequencies\n", populationCode_to_geneFrequencies)

populationCode_to_geneFrequencies
 {'GBR': [27, 53, 20], 'FIN': [39, 43, 23], 'CHS': [6, 34, 65], 'PUR': [21, 60, 23], 'CDX': [10, 47, 43], 'CLM': [19, 50, 26], 'IBS': [27, 55, 25], 'PEL': [7, 38, 40], 'PJL': [16, 48, 32], 'KHV': [6, 34, 59], 'ACB': [61, 34, 2], 'GWD': [82, 26, 5], 'ESN': [73, 25, 2], 'BEB': [9, 31, 46], 'MSL': [69, 19, 2], 'STU': [12, 44, 46], 'ITU': [9, 54, 39], 'CEU': [21, 61, 17], 'YRI': [73, 33, 1], 'CHB': [7, 29, 70], 'JPT': [5, 36, 64], 'LWK': [83, 18, 2], 'ASW': [30, 27, 4], 'MXL': [14, 26, 24], 'TSI': [21, 53, 37], 'GIH': [17, 53, 35]}


In [116]:
def print_populationCode_to_geneFrequencies(populationCode_to_geneFrequencies):
    print("PopCode --  homozygousRecessive, heterozygous, homozygousDominate")
    for key, value in populationCode_to_geneFrequencies.items():
        print(key, " ----- ", value[0], value[1], value[2])

In [117]:
print_populationCode_to_geneFrequencies(populationCode_to_geneFrequencies)

PopCode --  homozygousRecessive, heterozygous, homozygousDominate
GBR  -----  27 53 20
FIN  -----  39 43 23
CHS  -----  6 34 65
PUR  -----  21 60 23
CDX  -----  10 47 43
CLM  -----  19 50 26
IBS  -----  27 55 25
PEL  -----  7 38 40
PJL  -----  16 48 32
KHV  -----  6 34 59
ACB  -----  61 34 2
GWD  -----  82 26 5
ESN  -----  73 25 2
BEB  -----  9 31 46
MSL  -----  69 19 2
STU  -----  12 44 46
ITU  -----  9 54 39
CEU  -----  21 61 17
YRI  -----  73 33 1
CHB  -----  7 29 70
JPT  -----  5 36 64
LWK  -----  83 18 2
ASW  -----  30 27 4
MXL  -----  14 26 24
TSI  -----  21 53 37
GIH  -----  17 53 35


In [118]:
def total(populationCode_to_geneFrequencies):
    totalList = []
    for value in populationCode_to_geneFrequencies.values():
        total = value[0] + value[1] + value[2]
        totalList.append(total)

    # print(totalList)
    return totalList

In [119]:
totalGeneFrequencies = total(populationCode_to_geneFrequencies)

In [120]:
def get_populationCode_to_genePercentages(populationCode_to_geneFrequencies, totalGeneFrequencies):

    populationCode_to_genePercentages = {}

    cnt = 0
    # print(totalGeneFrequencies[int(cnt)])
    for key, value in populationCode_to_geneFrequencies.items():
        totalDivider = totalGeneFrequencies[int(cnt)]
        cnt += 1
        homozygousRecessive = value[0]/totalDivider
        heterozygous = value[1]/totalDivider
        homozygousDominate = value[2]/totalDivider

        populationCode_to_genePercentages[key] = [homozygousRecessive, heterozygous, homozygousDominate]

    return populationCode_to_genePercentages


In [121]:
populationCode_to_genePercentages = get_populationCode_to_genePercentages(populationCode_to_geneFrequencies, totalGeneFrequencies)
# print(populationCode_to_genePercentages)

In [122]:
def print_populationCode_to_genePercentages(populationCode_to_genePercentages):
    print("PopCode --  homozygousRecessive, heterozygous, homozygousDominate")
    for key, value in populationCode_to_genePercentages.items():
        print(key, " ----- ", format(value[0], '.3f'), format(value[1], '.3f'), format(value[2], '.3f'))

In [123]:
print_populationCode_to_genePercentages(populationCode_to_genePercentages)

PopCode --  homozygousRecessive, heterozygous, homozygousDominate
GBR  -----  0.270 0.530 0.200
FIN  -----  0.371 0.410 0.219
CHS  -----  0.057 0.324 0.619
PUR  -----  0.202 0.577 0.221
CDX  -----  0.100 0.470 0.430
CLM  -----  0.200 0.526 0.274
IBS  -----  0.252 0.514 0.234
PEL  -----  0.082 0.447 0.471
PJL  -----  0.167 0.500 0.333
KHV  -----  0.061 0.343 0.596
ACB  -----  0.629 0.351 0.021
GWD  -----  0.726 0.230 0.044
ESN  -----  0.730 0.250 0.020
BEB  -----  0.105 0.360 0.535
MSL  -----  0.767 0.211 0.022
STU  -----  0.118 0.431 0.451
ITU  -----  0.088 0.529 0.382
CEU  -----  0.212 0.616 0.172
YRI  -----  0.682 0.308 0.009
CHB  -----  0.066 0.274 0.660
JPT  -----  0.048 0.343 0.610
LWK  -----  0.806 0.175 0.019
ASW  -----  0.492 0.443 0.066
MXL  -----  0.219 0.406 0.375
TSI  -----  0.189 0.477 0.333
GIH  -----  0.162 0.505 0.333


# Conduct statistical tests on population code to GeneFrequencies 

- test with individuals
- you want counts, not frequencies
- test with continental groups 
- use chi square test 

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

In [131]:
def create_df_from_populationCode_to_geneFrequencies(populationCode_to_geneFrequencies):
    
    geneFrequenciesList = []
    populationCodesList = []
    for key, value in populationCode_to_geneFrequencies.items():
        geneFrequenciesList.append(value)
        populationCodesList.append(key)

    # Create a list of column names
    columns = ['Recesssive', 'Heterozygous', 'Dominant']

    # Convert the list of lists to a DataFrame
    df = pd.DataFrame(geneFrequenciesList, columns=columns, index=populationCodesList)

    # Print the resulting DataFrame
    # print(df)

    return df

In [132]:
dataframe_of_populationCode_to_geneFrequencies = create_df_from_populationCode_to_geneFrequencies(populationCode_to_geneFrequencies)
print(dataframe_of_populationCode_to_geneFrequencies)

     Recesssive  Heterozygous  Dominant
GBR          27            53        20
FIN          39            43        23
CHS           6            34        65
PUR          21            60        23
CDX          10            47        43
CLM          19            50        26
IBS          27            55        25
PEL           7            38        40
PJL          16            48        32
KHV           6            34        59
ACB          61            34         2
GWD          82            26         5
ESN          73            25         2
BEB           9            31        46
MSL          69            19         2
STU          12            44        46
ITU           9            54        39
CEU          21            61        17
YRI          73            33         1
CHB           7            29        70
JPT           5            36        64
LWK          83            18         2
ASW          30            27         4
MXL          14            26        24


In [136]:
def perform_chi_square_test(df):
    # Perform the chi-square test
    chi2, p, dof, expected = chi2_contingency(df)

    # Print the results
    print(f'Chi-square statistic: {chi2:.3f}')
    print(f'p-value: {p:.3f}')
    print(f'p-value:', p)
    print(f'Degrees of freedom: {dof}')
    print('Expected frequencies:')
    print(expected)

    return chi2

In [137]:
chi2 = perform_chi_square_test(dataframe_of_populationCode_to_geneFrequencies)

Chi-square statistic: 1016.589
p-value: 0.000
p-value: 2.6643478095338636e-180
Degrees of freedom: 50
Expected frequencies:
[[29.99607381 40.4789949  29.52493129]
 [31.4958775  42.50294464 31.00117786]
 [31.4958775  42.50294464 31.00117786]
 [31.19591676 42.09815469 30.70592854]
 [29.99607381 40.4789949  29.52493129]
 [28.49627012 38.45504515 28.04868473]
 [32.09579898 43.31252454 31.59167648]
 [25.49666274 34.40714566 25.0961916 ]
 [28.79623086 38.8598351  28.34393404]
 [29.69611307 40.07420495 29.22968198]
 [29.0961916  39.26462505 28.63918335]
 [33.89556341 45.74126423 33.36317236]
 [29.99607381 40.4789949  29.52493129]
 [25.79662348 34.81193561 25.39144091]
 [26.99646643 36.43109541 26.57243816]
 [30.59599529 41.28857479 30.11542992]
 [30.59599529 41.28857479 30.11542992]
 [29.69611307 40.07420495 29.22968198]
 [32.09579898 43.31252454 31.59167648]
 [31.79583824 42.90773459 31.29642717]
 [31.4958775  42.50294464 31.00117786]
 [30.89595603 41.69336474 30.41067923]
 [18.29760503 24.6

1. Gather a large amount of SNP data from the same chromosome as the desired SNP
2. Calculate chi square stat for each snp and store that in a list
3. plot that list of chi square states in a histogram for which i compared the desired SNP chi square stat against to determine significant